# Day 3
## Part 1

In [27]:
open System

type NumberRun = { Row : int; StartIndex : int; EndIndex : int; Value : string }

let numberRuns row line =
    let rec numberRuns text index runs run =
        match run with
        | None ->
            match text with
            | [] -> runs
            | c::rest ->
                if Char.IsDigit c then
                    numberRuns rest (index+1) runs (Some { Row = row; StartIndex = index; EndIndex = index; Value = string c })
                else
                    numberRuns rest (index+1) runs None
        | Some run ->
            match text with
            | [] -> run :: runs
            | c::rest ->
                if Char.IsDigit c then
                    numberRuns rest (index+1) runs (Some { run with EndIndex = index; Value = run.Value + string c })
                else
                    numberRuns rest (index+1) (run::runs) None
    numberRuns (List.ofSeq line) 0 [] None

type Symbol = { Row : int; Column : int; Value : char }

let symbolPositions row line =
    line 
    |> Seq.indexed 
    |> Seq.choose (fun (i, c) -> if not (c = '.' || Char.IsDigit c) then Some { Row = row; Column = i; Value = c } else None)
    |> List.ofSeq

let isAdjacent (number : NumberRun) (symbol : Symbol) =
    match abs (number.Row - symbol.Row) with
    | 0 ->
        number.StartIndex = symbol.Column + 1 || number.EndIndex = symbol.Column - 1
    | 1 ->
        number.StartIndex - 1 <= symbol.Column && number.EndIndex + 1 >= symbol.Column
    | _ -> false

let allNumbers lines = lines |> Seq.mapi numberRuns |> Seq.concat |> List.ofSeq
let allSymbols lines = lines |> Seq.mapi symbolPositions |> Seq.concat |> List.ofSeq

let sumAttached lines =
    let numbers = allNumbers lines
    let symbols = allSymbols lines
    let attached = numbers |> List.filter (fun n -> symbols |> List.exists (isAdjacent n)) 
    attached |> List.sumBy (fun n -> int n.Value)


In [28]:
#r "nuget:FsUnit.Xunit"
open FsUnitTyped

let testData = 
    [
        "467..114.."
        "...*......"
        "..35..633."
        "......#..."
        "617*......"
        ".....+.58."
        "..592....."
        "......755."
        "...$.*...."
        ".664.598.."
    ]

let testResult = sumAttached testData
testResult |> shouldEqual 4361

Installed Packages FsUnit.Xunit, 5.6.1

In [29]:
open System.IO
let input = File.ReadAllLines "input_03.txt"

let result1 =
    input
    |> sumAttached


In [33]:
printfn "Part 1: %d" result1
result1 |> shouldEqual 527369 // In case I break it

Part 1: 527369


## Part 2

In [32]:
let sumGearRatios lines =
    let numbers = allNumbers lines
    let symbols = allSymbols lines
    let gearRatios = 
        symbols 
        |> List.filter (fun s -> s.Value = '*')
        |> List.map (fun gear -> numbers |> List.filter (fun number -> isAdjacent number gear))
        |> List.choose (function | [x;y] -> Some (int x.Value * int y.Value) | _ -> None )

    List.sum gearRatios


In [35]:
let result2 = sumGearRatios input
printf "Result2 = %d" result2
result2 |> shouldEqual 73074886 // In case I break it


Result2 = 73074886